In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import pymongo
from pymongo import MongoClient

file_name = "../geotagged_tweets_20160812-0912.jsons"

In [13]:
user_cols = ['user.name', 'user.id_str', 'user.location', 'user.followers_count']

place_cols = ['place.id', 'place.url', 'place.place_type', 'place.name', 'place.full_name', 
              'place.country_code', 'place.country', 'place.bounding_box.type', 
              'place.bounding_box.coordinates']

entities_cols = ['entities.hashtags', 'entities.urls', 'entities.user_mentions', 'entities.symbols']

other_cols = ['created_at', 'text', 'truncated', 'retweet_count', 'favorite_count', 
               'filter_level', 'lang', 'timestamp_ms']

useful_cols = other_cols + user_cols + place_cols + entities_cols

In [2]:
client = MongoClient()
db = client.twitter
collection = db.inventory
data = pd.DataFrame(list(collection.find()))

KeyError: "['user.location', 'place.place_type', 'user.followers_count', 'place.country_code', 'entities.hashtags', 'place.bounding_box.type', 'place.bounding_box.coordinates', 'place.url', 'user.id_str', 'place.id', 'place.name', 'user.name', 'entities.symbols', 'place.country', 'place.full_name', 'entities.user_mentions', 'entities.urls'] not in index"

In [26]:
data['user'] = data['user'].apply(lambda x: {} if pd.isna(x) else x)
user = json_normalize(data['user'], errors = 'ignore')
user.columns = ['user.' + str(col) for col in user.columns]
user = user[user_cols]

print("USER")

data['place'] = data['place'].apply(lambda x: {} if pd.isna(x) else x)
place = json_normalize(data['place'], errors = 'ignore')
place.columns = ['place.' + str(col) for col in place.columns]
place = place[place_cols]

print("PLACE")

data['entities'] = data['entities'].apply(lambda x: {} if pd.isna(x) else x)
entities = json_normalize(data['entities'], errors = 'ignore')
entities.columns = ['entities.' + str(col) for col in entities.columns]
entities = entities[entities_cols]

print("ENTITIES")

USER
PLACE
ENTITIES


In [27]:
data = data.drop(columns=['user', 'place', 'entities'])
data = pd.concat([data, user, place, entities], axis=1, sort=False)[useful_cols]
data.head()

,created_at,text,truncated,retweet_count,favorite_count,filter_level,lang,timestamp_ms,user.name,user.id_str,...,place.name,place.full_name,place.country_code,place.country,place.bounding_box.type,place.bounding_box.coordinates,entities.hashtags,entities.urls,entities.user_mentions,entities.symbols
0,Fri Aug 12 10:04:00 +0000 2016,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,False,0,0,low,und,1470996240225,GIL DUPUY,366636488,...,Frontenac,"Frontenac, MO",US,United States,Polygon,"[[[-90.433909, 38.606163], [-90.433909, 38.647...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",[]
1,Fri Aug 12 10:04:02 +0000 2016,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,False,0,0,low,en,1470996242194,Red Octopus,82496193,...,Baton Rouge,"Baton Rouge, LA",US,United States,Polygon,"[[[-91.218994, 30.354161], [-91.218994, 30.564...","[{'text': 'NOJUSTICE', 'indices': [62, 72]}, {...","[{'url': 'https://t.co/5GMNZq40V3', 'expanded_...","[{'screen_name': 'BarackObama', 'name': 'Barac...",[]
2,Fri Aug 12 10:04:21 +0000 2016,@HillaryClinton he will do in one year all the...,False,0,0,low,en,1470996261125,JanJorgenson,44032624,...,Melbourne,"Melbourne, Victoria",AU,Australia,Polygon,"[[[144.593742, -38.433859], [144.593742, -37.5...",[],[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[]
3,Fri Aug 12 10:04:10 +0000 2016,@theblaze @realDonaldTrump https://t.co/n050DB...,False,0,0,low,und,1470996250817,GIL DUPUY,366636488,...,Frontenac,"Frontenac, MO",US,United States,Polygon,"[[[-90.433909, 38.606163], [-90.433909, 38.647...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",[]
4,Fri Aug 12 10:04:30 +0000 2016,#CNN #newday clear #Trump deliberately throwin...,False,0,0,low,en,1470996270035,Beverly Spence,769208504,...,Baltimore,"Baltimore, MD",US,United States,Polygon,"[[[-76.711521, 39.197211], [-76.711521, 39.372...","[{'text': 'CNN', 'indices': [0, 4]}, {'text': ...",[],[],[]


In [28]:
data

,created_at,text,truncated,retweet_count,favorite_count,filter_level,lang,timestamp_ms,user.name,user.id_str,...,place.name,place.full_name,place.country_code,place.country,place.bounding_box.type,place.bounding_box.coordinates,entities.hashtags,entities.urls,entities.user_mentions,entities.symbols
0,Fri Aug 12 10:04:00 +0000 2016,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,False,0,0,low,und,1470996240225,GIL DUPUY,366636488,...,Frontenac,"Frontenac, MO",US,United States,Polygon,"[[[-90.433909, 38.606163], [-90.433909, 38.647...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",[]
1,Fri Aug 12 10:04:02 +0000 2016,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,False,0,0,low,en,1470996242194,Red Octopus,82496193,...,Baton Rouge,"Baton Rouge, LA",US,United States,Polygon,"[[[-91.218994, 30.354161], [-91.218994, 30.564...","[{'text': 'NOJUSTICE', 'indices': [62, 72]}, {...","[{'url': 'https://t.co/5GMNZq40V3', 'expanded_...","[{'screen_name': 'BarackObama', 'name': 'Barac...",[]
2,Fri Aug 12 10:04:21 +0000 2016,@HillaryClinton he will do in one year all the...,False,0,0,low,en,1470996261125,JanJorgenson,44032624,...,Melbourne,"Melbourne, Victoria",AU,Australia,Polygon,"[[[144.593742, -38.433859], [144.593742, -37.5...",[],[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[]
3,Fri Aug 12 10:04:10 +0000 2016,@theblaze @realDonaldTrump https://t.co/n050DB...,False,0,0,low,und,1470996250817,GIL DUPUY,366636488,...,Frontenac,"Frontenac, MO",US,United States,Polygon,"[[[-90.433909, 38.606163], [-90.433909, 38.647...",[],[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze...",[]
4,Fri Aug 12 10:04:30 +0000 2016,#CNN #newday clear #Trump deliberately throwin...,False,0,0,low,en,1470996270035,Beverly Spence,769208504,...,Baltimore,"Baltimore, MD",US,United States,Polygon,"[[[-76.711521, 39.197211], [-76.711521, 39.372...","[{'text': 'CNN', 'indices': [0, 4]}, {'text': ...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657302,Mon Sep 12 13:20:33 +0000 2016,@CarolCNN if MSM were honest watch any utube v...,False,0,0,low,en,1473686433874,Karen B,716017946166857728,...,Coral Gables,"Coral Gables, FL",US,United States,Polygon,"[[[-80.297165, 25.627038], [-80.297165, 25.771...",[],[],"[{'screen_name': 'CarolCNN', 'name': 'Carol Co...",[]
657303,Mon Sep 12 13:20:36 +0000 2016,#EEUU\nNeumonía obliga a @HillaryClinton a can...,False,0,0,low,es,1473686436682,La Razon Digital,44489439,...,Bolivia,Bolivia,BO,Bolivia,Polygon,"[[[-69.640762, -22.897682], [-69.640762, -9.66...","[{'text': 'EEUU', 'indices': [0, 5]}]","[{'url': 'https://t.co/ovXe59kn0n', 'expanded_...","[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[]
657304,Mon Sep 12 13:20:38 +0000 2016,It's interesting that Hillary Clinton's crowds...,False,0,0,low,en,1473686438961,Robert Chaffin,3241116564,...,Fairbanks,"Fairbanks, AK",US,United States,Polygon,"[[[-147.81382, 64.810474], [-147.81382, 64.865...",[],"[{'url': 'https://t.co/lrq2SOOwSO', 'expanded_...",[],[]
657305,Mon Sep 12 13:20:32 +0000 2016,"TRUMP U, TAXES ,WEIRD MEDICAL REPORT WITH A WH...",False,0,0,low,en,1473686432363,Sheryl Berghoff,569487350,...,San Diego,"San Diego, CA",US,United States,Polygon,"[[[-117.282538, 32.53962], [-117.282538, 33.08...",[],"[{'url': 'https://t.co/2qDahl7A9n', 'expanded_...",[],[]


,user.id,user.id_str,user.name,user.screen_name,user.location,user.url,user.description,user.protected,user.verified,user.followers_count,...,user.profile_text_color,user.profile_use_background_image,user.profile_image_url,user.profile_image_url_https,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.profile_banner_url
0,366636488,366636488,GIL DUPUY,DUPUY77,Miami,http://ggm-dupuy.com,"Fashion photographer, love action and adventur...",False,False,186,...,333333,True,http://pbs.twimg.com/profile_images/7609156129...,https://pbs.twimg.com/profile_images/760915612...,False,False,None,None,None,NaN
1,82496193,82496193,Red Octopus,redoctapus,"Ogletown, DE",None,"AWSCWI Pipefitter USAF NRA I remember USA, Hun...",False,False,531,...,000000,False,http://pbs.twimg.com/profile_images/6883746403...,https://pbs.twimg.com/profile_images/688374640...,False,False,None,None,None,https://pbs.twimg.com/profile_banners/82496193...
2,44032624,44032624,JanJorgenson,jjorgi30,Australia,None,The best movies are written and directed by pe...,False,False,180,...,3E4415,True,http://pbs.twimg.com/profile_images/7200177636...,https://pbs.twimg.com/profile_images/720017763...,False,False,None,None,None,https://pbs.twimg.com/profile_banners/44032624...
3,366636488,366636488,GIL DUPUY,DUPUY77,Miami,http://ggm-dupuy.com,"Fashion photographer, love action and adventur...",False,False,186,...,333333,True,http://pbs.twimg.com/profile_images/7609156129...,https://pbs.twimg.com/profile_images/760915612...,False,False,None,None,None,NaN
4,769208504,769208504,Beverly Spence,bspence5,None,None,"Retired, political wonk mother of one grandmot...",False,False,2652,...,333333,True,http://pbs.twimg.com/profile_images/3108377856...,https://pbs.twimg.com/profile_images/310837785...,False,False,None,None,None,NaN


In [ ]:
df1["created_at"] = pd.to_datetime(df1["created_at"], format="")